In [25]:
import pandas as pd
import numpy as np
from datasist.structdata import detect_outliers

from imblearn.over_sampling import SMOTE

from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

from sklearn.ensemble import (
    BaggingClassifier,
    ExtraTreesClassifier,
    RandomForestClassifier,
    StackingClassifier,
    HistGradientBoostingClassifier
)
from xgboost import XGBClassifier

import joblib ## for model exportation
from sklearn.ensemble import RandomForestClassifier

from sklearn.linear_model import LogisticRegression



In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)

df = pd.read_csv("cleaned_data.csv", low_memory=False)
df

,Age,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Total_EMI_per_month,Amount_invested_monthly,Monthly_Balance,Credit_Score,Credit-Builder Loan,Personal Loan,Debt Consolidation Loan,Student Loan,Payday Loan,Mortgage Loan,Auto Loan,Home Equity Loan,Month_August,Month_February,Month_January,Month_July,Month_June,Month_March,Month_May,Occupation_Architect,Occupation_Developer,Occupation_Doctor,Occupation_Engineer,Occupation_Entrepreneur,Occupation_Journalist,Occupation_Lawyer,Occupation_Manager,Occupation_Mechanic,Occupation_Media_Manager,Occupation_Musician,Occupation_Scientist,Occupation_Teacher,Occupation_Writer,Payment_of_Min_Amount_Yes,Payment_Behaviour_High_spent_Medium_value_payments,Payment_Behaviour_High_spent_Small_value_payments,Payment_Behaviour_Low_spent_Large_value_payments,Payment_Behaviour_Low_spent_Medium_value_payments,Payment_Behaviour_Low_spent_Small_value_payments
0,23.0,19114.12,1824.843333,3.0,4.0,3.0,4.0,3.0,7.0,11.27,4.0,1.0,809.98,26.822620,265.0,49.574949,80.415295,312.494089,2,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False
1,23.0,19114.12,3093.745000,3.0,4.0,3.0,4.0,-1.0,14.0,11.27,4.0,2.0,809.98,31.944960,219.0,49.574949,118.280222,284.629162,2,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False
2,-500.0,19114.12,3093.745000,3.0,4.0,3.0,4.0,3.0,7.0,9.40,4.0,2.0,809.98,28.609352,267.0,49.574949,81.699521,331.209863,2,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False
3,23.0,19114.12,3093.745000,3.0,4.0,3.0,4.0,5.0,4.0,6.27,4.0,2.0,809.98,31.377862,268.0,49.574949,199.458074,223.451310,2,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True
4,23.0,19114.12,1824.843333,3.0,4.0,3.0,4.0,6.0,14.0,11.27,4.0,2.0,809.98,24.797347,269.0,49.574949,41.420153,341.489231,2,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,25.0,39628.99,3359.415833,4.0,6.0,7.0,2.0,23.0,7.0,11.50,3.0,1.0,502.38,34.663572,378.0,35.104023,60.971333,479.866228,0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
99996,25.0,39628.99,3359.415833,4.0,6.0,7.0,2.0,18.0,7.0,11.50,3.0,1.0,502.38,40.565631,379.0,35.104023,54.185950,496.651610,0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False
99997,25.0,39628.99,3359.415833,4.0,6.0,13.0,2.0,27.0,6.0,11.50,3.0,2.0,502.38,41.255522,380.0,35.104023,24.028477,516.809083,0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
99998,25.0,39628.99,3359.415833,4.0,6.0,7.0,2.0,20.0,14.0,11.50,3.0,2.0,502.38,33.638208,381.0,35.104023,251.672582,319.164979,1,0.0,0.0,0.0,1.0,0.0,

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 54 columns):
 #   Column                                              Non-Null Count   Dtype  
---  ------                                              --------------   -----  
 0   Age                                                 100000 non-null  float64
 1   Annual_Income                                       100000 non-null  float64
 2   Monthly_Inhand_Salary                               100000 non-null  float64
 3   Num_Bank_Accounts                                   100000 non-null  float64
 4   Num_Credit_Card                                     100000 non-null  float64
 5   Interest_Rate                                       100000 non-null  float64
 6   Num_of_Loan                                         100000 non-null  float64
 7   Delay_from_due_date                                 100000 non-null  float64
 8   Num_of_Delayed_Payment                              100000 non-nu

In [5]:
X, y = df.drop("Credit_Score",axis=1).values , df["Credit_Score"] 
rus = SMOTE(sampling_strategy='auto')
X_data_rus, y_data_rus = rus.fit_resample(X, y)
y_data_rus.value_counts(normalize=True)

2    0.333333
1    0.333333
0    0.333333
Name: Credit_Score, dtype: float64

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X_data_rus, y_data_rus, test_size=0.2, 
                                                    random_state=42, stratify=y_data_rus)

from sklearn.preprocessing import PowerTransformer
scalar = PowerTransformer(method="yeo-johnson", standardize=True).fit(X_train)
X_train = scalar.transform(X_train)
X_test = scalar.transform(X_test)

In [9]:
# SGD Classifier
sgd_classifier = SGDClassifier(random_state=42)

# Hyperparameter tuning
sgd_param_grid = {
    'alpha': [0.0001, 0.001, 0.01, 0.1],
    'learning_rate': ['constant', 'optimal', 'invscaling'],
    'max_iter': [1000, 2000, 3000, 4000],  # Increase the max_iter values
    'eta0': [0.01, 0.1, 0.5]
}


grid_search_sgd = GridSearchCV(sgd_classifier, sgd_param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_sgd.fit(X_train, y_train)

# Print results
print("\nHyperparameter tuning for SGD Classifier:")
print(f"Best hyperparameters: {grid_search_sgd.best_params_}")
best_model_sgd = grid_search_sgd.best_estimator_

# Training score
training_score_sgd = best_model_sgd.score(X_train, y_train)
print(f"Training Score: {training_score_sgd}")

# Test score
y_pred_sgd = best_model_sgd.predict(X_test)
test_score_sgd = accuracy_score(y_test, y_pred_sgd)
print(f"Test Accuracy: {test_score_sgd}")

# Mean cross-validation score
cross_val_scores_sgd = cross_val_score(best_model_sgd, X, y, cv=5)
mean_cv_score_sgd = cross_val_scores_sgd.mean()
print(f"Mean Cross-Validation Score: {mean_cv_score_sgd}")

# joblib.dump(best_model_sgd, 'sgd_classifier_model.pkl')


Hyperparameter tuning for SGD Classifier:
Best hyperparameters: {'alpha': 0.0001, 'eta0': 0.1, 'learning_rate': 'invscaling', 'max_iter': 1000}
Training Score: 0.64
Test Accuracy: 0.63495
Mean Cross-Validation Score: 0.20068000000000003


['sgd_classifier_model.pkl']

In [13]:
kneighbors_classifier = KNeighborsClassifier()

# Hyperparameter tuning
# kneighbors_param_grid = {
#     'n_neighbors': [3, 5, 7, 9]
# }

kneighbors_param_grid = {
    'n_neighbors': [7, 9, 11, 13, 15],  # Fine-tune 'n_neighbors'
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto'],
    'p': [1, 2],
    'leaf_size': [10, 20, 30]
}


grid_search_kneighbors = GridSearchCV(kneighbors_classifier, kneighbors_param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_kneighbors.fit(X_train, y_train)

# Print results
print("\nHyperparameter tuning for K-Nearest Neighbors:")
print(f"Best hyperparameters: {grid_search_kneighbors.best_params_}")
best_model_kneighbors = grid_search_kneighbors.best_estimator_

# Training score
training_score_kneighbors = best_model_kneighbors.score(X_train, y_train)
print(f"Training Score: {training_score_kneighbors}")

# Test score
y_pred_kneighbors = best_model_kneighbors.predict(X_test)
test_score_kneighbors = accuracy_score(y_test, y_pred_kneighbors)
print(f"Test Accuracy: {test_score_kneighbors}")

# Mean cross-validation score
cross_val_scores_kneighbors = cross_val_score(best_model_kneighbors, X, y, cv=5)
mean_cv_score_kneighbors = cross_val_scores_kneighbors.mean()
print(f"Mean Cross-Validation Score: {mean_cv_score_kneighbors}")

# joblib.dump(best_model_kneighbors, 'kneighbors_classifier_model.pkl')





Hyperparameter tuning for K-Nearest Neighbors:
Best hyperparameters: {'algorithm': 'auto', 'leaf_size': 10, 'n_neighbors': 7, 'p': 1, 'weights': 'distance'}
Training Score: 1.0
Test Accuracy: 0.76195
Mean Cross-Validation Score: 0.52657


C:\Users\Naradon\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
80 fits failed out of a total of 320.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
80 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Naradon\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Naradon\anaconda3\Lib\site-packages\sklearn\svm\_classes.py", line 274, in fit
    self.coef_, self.intercept_, n_iter_ = _fit_liblinear(
                                           ^^^^^^^^^^^^^^^
  File "C:\Users\Naradon\anaconda3\Lib\site-packages\sklearn\svm\_base.py", line 1223, in _


Hyperparameter tuning for Linear SVC:
Best hyperparameters: {'C': 0.1, 'dual': False, 'max_iter': 1000, 'penalty': 'l1'}
Training Score: 0.6345
Test Accuracy: 0.62935


C:\Users\Naradon\anaconda3\Lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\Naradon\anaconda3\Lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\Naradon\anaconda3\Lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\Naradon\anaconda3\Lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Mean Cross-Validation Score: 0.6315999999999999


C:\Users\Naradon\anaconda3\Lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


['linear_svc_model.pkl']

In [ ]:
# # linear_svc_classifier = LinearSVC(random_state=42)
# ##############################################################################################################
# # Hyperparameter tuning
# linear_svc_param_grid = {
#     'C': [0.1, 1, 10, 100],
#     'max_iter': [1000, 2000, 3000, 4000],
#     'penalty': ['l1', 'l2'],
#     'dual': [True, False]
# }


# grid_search_linear_svc = GridSearchCV(linear_svc_classifier, linear_svc_param_grid, cv=5, scoring='accuracy', n_jobs=-1)
# grid_search_linear_svc.fit(X_train, y_train)

# # Print results
# print("\nHyperparameter tuning for Linear SVC:")
# print(f"Best hyperparameters: {grid_search_linear_svc.best_params_}")
# best_model_linear_svc = grid_search_linear_svc.best_estimator_

# # Training score
# training_score_linear_svc = best_model_linear_svc.score(X_train, y_train)
# print(f"Training Score: {training_score_linear_svc}")

# # Test score
# y_pred_linear_svc = best_model_linear_svc.predict(X_test)
# test_score_linear_svc = accuracy_score(y_test, y_pred_linear_svc)
# print(f"Test Accuracy: {test_score_linear_svc}")

# # Mean cross-validation score
# cross_val_scores_linear_svc = cross_val_score(best_model_linear_svc, X, y, cv=5)
# mean_cv_score_linear_svc = cross_val_scores_linear_svc.mean()
# print(f"Mean Cross-Validation Score: {mean_cv_score_linear_svc}")

# # joblib.dump(best_model_linear_svc, 'linear_svc_model.pkl')


In [7]:
# Best hyperparameters: {'C': 0.1, 'dual': False, 'max_iter': 1000, 'penalty': 'l1'}

LinearSVC_Model = LinearSVC(C=0.1, dual=False, max_iter = 10000, penalty = 'l1')
LinearSVC_Model.fit(X_train, y_train)

y_train_pred = LinearSVC_Model.predict(X_train)

train_accuracy = accuracy_score(y_train, y_train_pred)
print("Training Accuracy:", train_accuracy)

y_test_pred = LinearSVC_Model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print("Test Accuracy:", test_accuracy)

cv_scores = cross_val_score(LinearSVC_Model, X, y, cv=5, n_jobs=-1)
# print("Cross-validation Scores:", cv_scores)
print("Mean Cross-validation Score:", cv_scores.mean())

# joblib.dump(LinearSVC_Model, 'linear_svc_model.pkl')

Training Accuracy: 0.6344875
Test Accuracy: 0.62935
Cross-validation Scores: [0.62495 0.6358  0.6315  0.62715 0.6387 ]
Mean Cross-validation Score: 0.6316200000000001


In [9]:
RFmodel = RandomForestClassifier()
RFmodel.fit(X_train, y_train)

y_train_pred = RFmodel.predict(X_train)

train_accuracy = accuracy_score(y_train, y_train_pred)
print("Training Accuracy:", train_accuracy)

y_test_pred = RFmodel.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print("Test Accuracy:", test_accuracy)

cv_scores = cross_val_score(RFmodel, X, y, cv=5, n_jobs=-1)
# print("Cross-validation Scores:", cv_scores)
print("Mean Cross-validation Score:", cv_scores.mean())

# joblib.dump(RFmodel, 'RandomForest.pkl')

Training Accuracy: 1.0
Test Accuracy: 0.78975
Mean Cross-validation Score: 0.68941


In [11]:
BaggingModel = BaggingClassifier(n_jobs=-1)
BaggingModel.fit(X_train, y_train)

y_train_pred = BaggingModel.predict(X_train)

train_accuracy = accuracy_score(y_train, y_train_pred)
print("Training Accuracy:", train_accuracy)

y_test_pred = BaggingModel.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print("Test Accuracy:", test_accuracy)

cv_scores = cross_val_score(BaggingModel, X, y, cv=5, n_jobs=-1)
# print("Cross-validation Scores:", cv_scores)
print("Mean Cross-validation Score:", cv_scores.mean())

# joblib.dump(BaggingModel, 'BaggingModel.pkl')

Training Accuracy: 0.986075
Test Accuracy: 0.7697
Mean Cross-validation Score: 0.66314


['BaggingModel.pkl']

In [12]:
ETmodel = ExtraTreesClassifier(max_depth=10, n_jobs=-1)
ETmodel.fit(X_train, y_train)

y_train_pred = ETmodel.predict(X_train)

train_accuracy = accuracy_score(y_train, y_train_pred)
print("Training Accuracy:", train_accuracy)

y_test_pred = ETmodel.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print("Test Accuracy:", test_accuracy)

cv_scores = cross_val_score(ETmodel, X, y, cv=5, n_jobs=-1)
# print("Cross-validation Scores:", cv_scores)
print("Mean Cross-validation Score:", cv_scores.mean())

# joblib.dump(ETmodel, 'ExtraTrees.pkl')

Training Accuracy: 0.699825
Test Accuracy: 0.6721
Mean Cross-validation Score: 0.6529299999999999


['ExtraTrees.pkl']

In [13]:
HistGradientBoosting = HistGradientBoostingClassifier()
HistGradientBoosting.fit(X_train, y_train)

y_train_pred = HistGradientBoosting.predict(X_train)

train_accuracy = accuracy_score(y_train, y_train_pred)
print("Training Accuracy:", train_accuracy)

y_test_pred = HistGradientBoosting.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print("Test Accuracy:", test_accuracy)

cv_scores = cross_val_score(HistGradientBoosting, X, y, cv=5, n_jobs=-1)
# print("Cross-validation Scores:", cv_scores)
print("Mean Cross-validation Score:", cv_scores.mean())

# joblib.dump(HistGradientBoosting, 'HistGradientBoosting.pkl')

Training Accuracy: 0.7768375
Test Accuracy: 0.73745
Mean Cross-validation Score: 0.6935800000000001


['HistGradientBoosting.pkl']

In [14]:
XGB = XGBClassifier(n_jobs=-1)
XGB.fit(X_train, y_train)

y_train_pred = XGB.predict(X_train)

train_accuracy = accuracy_score(y_train, y_train_pred)
print("Training Accuracy:", train_accuracy)

y_test_pred = XGB.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print("Test Accuracy:", test_accuracy)

cv_scores = cross_val_score(XGB, X, y, cv=5, n_jobs=-1)
# print("Cross-validation Scores:", cv_scores)
print("Mean Cross-validation Score:", cv_scores.mean())

joblib.dump(XGB, 'XGB.pkl')

Training Accuracy: 0.8228875
Test Accuracy: 0.75565
Mean Cross-validation Score: 0.68721


['XGB.pkl']

In [27]:
Bagging = BaggingClassifier(n_jobs=-1)
ET = ExtraTreesClassifier(max_depth=10, n_jobs=-1)
rForest = RandomForestClassifier(n_jobs=-1)
HistGradientBoosting = HistGradientBoostingClassifier()
XGB = XGBClassifier(n_jobs=-1)

meta_learner = LogisticRegression(max_iter=1000)

stackingModel = StackingClassifier([
    ('Bagging', Bagging),
    ('ET', ET),
    ('rForest', rForest),
    ('HistGradientBoosting', HistGradientBoosting),
    ('XGB', XGB)
], final_estimator=meta_learner, n_jobs=-1)

stackingModel.fit(X_train, y_train)



StackingClassifier(estimators=[('Bagging', BaggingClassifier(n_jobs=-1)),
                               ('ET',
                                ExtraTreesClassifier(max_depth=10, n_jobs=-1)),
                               ('rForest', RandomForestClassifier(n_jobs=-1)),
                               ('HistGradientBoosting',
                                HistGradientBoostingClassifier()),
                               ('XGB',
                                XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_by...
                                              interaction_constraints=None,
                                              learning_rate=None, max_bin=None,
                                              max_cat_threshold=None,
                                              max_cat_to_onehot=None,
                                              max_delta_step=None,
                                              max_depth=None, max_leaves=None,
                                              min_child_weight=None,
                                              missing=nan,
                                              monotone_constraints=None,
                                              multi_strategy=None,
                                              n_estimators=None, n_jobs=-1,
                                              num_parallel_tree=None,
                                              random_state=None, ...))],
                   final_estimator=LogisticRegression(max_iter=1000),
                   n_jobs=-1)

In [28]:
print("Train Score: ",stackingModel.score(X_train, y_train))

Train Score:  0.99995


In [29]:
print("Test Score: ",stackingModel.score(X_test, y_test))

Test Score:  0.79565


In [30]:
cv_accuracy = cross_val_score(stackingModel, X_train, y_train, cv=5, n_jobs=-1).mean()
print("Cross-Validation Accuracy:", cv_accuracy)

Cross-Validation Accuracy: 0.7856500000000001


In [26]:
knn_model = KNeighborsClassifier(algorithm='auto', 
                                 leaf_size=10, 
                                 n_neighbors=7, 
                                 p=1, 
                                 weights='distance')

# Fit the model to your training data
knn_model.fit(X_train, y_train)
train_predictions = knn_model.predict(X_train)
test_predictions = knn_model.predict(X_test)

# Calculate accuracy for training and testing data
train_accuracy = accuracy_score(y_train, train_predictions)
test_accuracy = accuracy_score(y_test, test_predictions)

print("Train Accuracy:", train_accuracy)
print("Test Accuracy:", test_accuracy)


# Export the trained model to a file
# joblib.dump(knn_model, 'knn_model.pkl')

Train Accuracy: 1.0
Test Accuracy: 0.76195


In [21]:
# joblib.dump(knn_model, 'knn_model.pkl')

['knn_model.pkl']